<a href="https://colab.research.google.com/github/charlie9526/TaxiFare/blob/master/second_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Inports and Mounting

In [488]:
from google.colab import drive 
drive.mount('/content/drive')

# best_rf_yet = {'esti__bootstrap': False,
#  'esti__max_depth': 67,
#  'esti__max_features': 'sqrt',
#  'esti__min_samples_leaf': 2,
#  'esti__min_samples_split': 5,
#  'esti__n_estimators': 1621}
#

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import pandas as pd
from datetime import datetime
import random
from google.colab import files
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns; sns.set(style="ticks", color_codes=True)              #data visualization library based on matplotlib
import numpy as np                  #support library for amtrices etc.
from scipy.stats import norm   
import warnings
warnings.filterwarnings('ignore')   #manage warnings
%matplotlib inline                  
#magic function in ipynb. ploting the graphs inline after the code
from math import sin, cos, sqrt, atan2, radians

from sklearn.svm import SVC,LinearSVC
from sklearn.decomposition import PCA
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import OneHotEncoder
from sklearn import metrics
from sklearn.metrics import f1_score
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder,LabelBinarizer
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.linear_model import LogisticRegression
from sklearn.multioutput import MultiOutputClassifier
from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import roc_curve, roc_auc_score
from sklearn.base import BaseEstimator, TransformerMixin

In [0]:
train_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/train.csv')
test_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/test.csv')
trip_id  = test_df['tripid']

sample_submission_df = pd.read_csv('/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/sample_submission.csv')



In [0]:
################### ugly indexes
train_df = train_df[(train_df['tripid']!=213112837) | (train_df['tripid']!=193653017)]

## Definitions

In [0]:
#elts write a lambda function
def get_duration(x):
  if x['pickup_time']==None or x['drop_time']==None:
    return None
  
  try :
    if int(x['pickup_time'].split("/")[1])>12 or int(x['drop_time'].split("/")[1])>12:
      gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
    else:
      if int(x['pickup_time'].split("/")[1]) != int(x['drop_time'].split("/")[1]):
        gap = datetime.strptime(x['pickup_time'],"%m/%d/%Y %H:%M")-datetime.strptime(x['drop_time'],"%m/%d/%Y %H:%M")
      else:
        gap = datetime.strptime(x['pickup_time'],"%d/%m/%Y %H:%M")-datetime.strptime(x['drop_time'],"%d/%m/%Y %H:%M")
    # print (gap.total_seconds()/3600,x['pickup_time'] ,x['drop_time'])
  except :
    # print(int(x['pickup_time'].split("/")[0])>12 or int(x['drop_time'].split("/")[0])>12)
    return None
  # print(abs(gap.total_seconds()/3600), x['drop_time'],x['pickup_time'])
  return abs(gap.total_seconds())

class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
        return self

    def transform(self, X):
        X['time_gap'] = X.apply(lambda x:get_duration(x), axis=1)
        X = X.drop(labels=['pickup_time','drop_time'],axis=1)
        return X

# Change the label of binary values
class MakeTripDurationFeature(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        y = y.apply(lambda x:get_binary(x))
        # print (y);
        return self
    def transform(self,X):
        return X

# debugerr 
class debugerr(BaseEstimator, TransformerMixin):
    def fit(self, X, y):
        print (pd.DataFrame(X));
        return self
    def transform(self,X):
        return X

# lambda function for change label
def get_binary(x):
  if x==None:
    return 0
  if x == "correct":
    # print (x);
    return 1
  else:
    return 0

# set distance column
class set_distance(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X['distance'] = X.apply(lambda x:get_distance(x), axis=1)
      # print(X)
      # X = X.drop(labels=['pick_lat','pick_lon','drop_lat','drop_lon'],axis=1)
      return X
# lambda column get distance
def get_distance(x):
  R = 6373.0
  lat1 = radians(float(x['drop_lat']))
  lon1 = radians(float(x['drop_lon']))
  lat2 = radians(float(x['pick_lat']))
  lon2 = radians(float(x['pick_lon']))

  dlon = lon2 - lon1
  dlat = lat2 - lat1

  a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
  c = 2 * atan2(sqrt(a), sqrt(1 - a))

  distance = R * c
  return distance

# remove columns
class remove_colums_unwanted(BaseEstimator, TransformerMixin):
    def fit(self, X, y=None):
      return self

    def transform(self, X):
      X = X.drop(labels=['pick_lat','pick_lon','meter_waiting_fare','drop_lat','drop_lon','pickup_time','drop_time'],axis=1)
      return X


def encoding_label(label):
  if(label=='correct'):
    return 1
  else:
    return 0

def get_hour_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  return datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").hour

################### Lets get date out as a column
################### lambda function

def get_date_value(x,date_time_column_name):
  if x[date_time_column_name]==None:
    return None
  month = str(datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").month)
  date = str(datetime.strptime(x[date_time_column_name],"%m/%d/%Y %H:%M").day)
  return int(month+date)

def calculate_direction(d_lon, d_lat):
    result = np.zeros(len(d_lon))
    l = np.sqrt(d_lon**2 + d_lat**2)
    result[d_lon>0] = (180/np.pi)*np.arcsin(d_lat[d_lon>0]/l[d_lon>0])
    idx = (d_lon<0) & (d_lat>0)
    result[idx] = 180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    idx = (d_lon<0) & (d_lat<0)
    result[idx] = -180 - (180/np.pi)*np.arcsin(d_lat[idx]/l[idx])
    return result


## Pre-processing

check null values

In [493]:
train_df['fare'].fillna(0,inplace=True)
test_df['fare'].fillna(0,inplace=True)
train_df['additional_fare'].fillna(0,inplace=True)
test_df['additional_fare'].fillna(0,inplace=True)

train_df.isna().sum()

tripid                         0
additional_fare                0
duration                     202
meter_waiting                202
meter_waiting_fare           202
meter_waiting_till_pickup    202
pickup_time                    0
drop_time                      0
pick_lat                       0
pick_lon                       0
drop_lat                       0
drop_lon                       0
fare                           0
label                          0
dtype: int64

In [494]:
train_df['fare']

0        270.32
1        197.85
2        301.64
3         82.30
4        358.39
          ...  
17171    198.26
17172    581.23
17173     76.20
17174    133.31
17175     98.57
Name: fare, Length: 17176, dtype: float64

encode label and create feature set


In [0]:
train_df['label'] = train_df['label'].apply(encoding_label).values
features_df = train_df.loc[:, train_df.columns != 'label']
label_df = train_df['label']

remove unnecessary columns

In [0]:
features_df = features_df.loc[:, features_df.columns != 'tripid']
test_df = test_df.loc[:, test_df.columns != 'tripid']

add some columns to dataset

In [0]:
train_df['delta_lat'] = train_df.pick_lat - train_df.drop_lat
train_df['delta_lon'] = train_df.pick_lon - train_df.drop_lon
train_df['direction'] = calculate_direction(train_df.delta_lon, train_df.delta_lat)

test_df['delta_lat'] = test_df.pick_lat - test_df.drop_lat
test_df['delta_lon'] = test_df.pick_lon - test_df.drop_lon
test_df['direction'] = calculate_direction(test_df.delta_lon, test_df.delta_lat)

features_df['pickup_hour'] = features_df.apply(lambda x:get_hour_value(x,"pickup_time"),axis=1)
# features_df['drop_hour'] = features_df.apply(lambda x:get_hour_value(x,"drop_time"),axis=1)


# features_df['duration'] = features_df.apply(lambda x:get_duration(x),axis=1)
# test_df['duration'] = test_df.apply(lambda x:get_duration(x),axis=1)

test_df['pickup_hour'] = test_df.apply(lambda x:get_hour_value(x,"pickup_time"),axis=1)
# test_df['drop_hour'] = test_df.apply(lambda x:get_hour_value(x,"drop_time"),axis=1)

features_df['distance'] = features_df.apply(lambda x:get_distance(x), axis=1)
test_df['distance'] = test_df.apply(lambda x:get_distance(x), axis=1)

features_df = features_df.drop(labels=['drop_lat','drop_lon','pickup_time','pick_lat','pick_lon','drop_time'],axis=1)
test_df = test_df.drop(labels=['drop_lat','drop_lon','pickup_time','pick_lat','pick_lon','drop_time'],axis=1)

features_df.to_csv("/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/features_df.csv", index=True)


split train and test data

In [0]:
X_train, X_test, y_train, y_test = train_test_split(features_df, label_df, test_size=0.3, random_state=1)


## Model Pipelines

In [499]:
# cat_cols=features_df.columns[features_df.dtypes == "object"].values
# cat_preprocessing_steps=Pipeline([
#   ('cat_imputer',SimpleImputer(strategy='constant', fill_value='missing')),
#   ('cat_one_hot',OneHotEncoder(handle_unknown='ignore'))
# ])

numeric_preprocessing_steps = Pipeline([
    ('standard_scaler', StandardScaler()),
    ('simple_imputer', SimpleImputer(strategy='mean'))
])

numeric_cols = features_df.columns[features_df.dtypes != "object"].values

preprocessor = ColumnTransformer(
    transformers = [
        ("numeric", numeric_preprocessing_steps, numeric_cols)
    ],
    remainder = "drop"
)
print(numeric_cols)

['additional_fare' 'duration' 'meter_waiting' 'meter_waiting_fare'
 'meter_waiting_till_pickup' 'fare' 'pickup_hour' 'distance']


create random grid search cv for random forest

In [0]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.ensemble import RandomForestRegressor



# # Number of trees in random forest
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 20)]
# # Number of features to consider at every split
max_features = ['auto', 'sqrt']
# Maximum number of levels in tree
max_depth = [int(x) for x in np.linspace(10, 110, num = 15)]
max_depth.append(None)
# # Minimum number of samples required to split a node
min_samples_split = [2, 5, 10, 20]
# # Minimum number of samples required at each leaf node
min_samples_leaf = [1, 2, 4 , 10]
# # Method of selecting samples for training each tree
bootstrap = [True, False]

random_grid_random_forest = {
               'esti__n_estimators': n_estimators,
               'esti__max_features': max_features,
               'esti__max_depth': max_depth,
               'esti__min_samples_split': min_samples_split,
               'esti__min_samples_leaf': min_samples_leaf,
               'esti__bootstrap': bootstrap
               }
random_grid_svm = {
              "esti__C": np.arange(2, 10, 2),
              "esti__gamma": np.arange(0.1, 1, 0.2)
              }

RF: 0.973418 (0.003592)

In [0]:

pca = PCA()
# estimators = SVC(C=8,gamma=0.7,kernel='rbf')

estimators = RandomForestClassifier(
      # bootstrap= False,
      max_depth=67,
      # max_features= 'sqrt',
      # min_samples_leaf= 2,
      # min_samples_split=5,
      # n_estimators = 1621
)
remove_c = remove_colums_unwanted()

full_pipeline = Pipeline([
                          ("preprocessor", preprocessor),
                          # ('columnremove',remove_c),
                          # ("debuffe",debugerr()),
                          # ("pca",pca),
                          ("esti", estimators)
                          ])



In [502]:
kfold = StratifiedKFold(n_splits=10, random_state=42, shuffle=True)
cv_results = cross_val_score(full_pipeline, features_df, label_df, cv=kfold, scoring='f1')
print('%s: %f (%f)' % ("RF", cv_results.mean(), cv_results.std()))

RF: 0.973418 (0.003592)


In [0]:
# rf_random_pipe = RandomizedSearchCV(full_pipeline, param_distributions = random_grid_svm, cv = 10, verbose=2, random_state=42, scoring ='f1')
# rf_random_pipe.fit(features_df,label_df)

In [0]:
full_pipeline.fit(features_df,label_df)
predicted = full_pipeline.predict(test_df)
data_dict = { 'tripid':trip_id, 'prediction':predicted}
pd_test_result = pd.DataFrame(data=data_dict)


In [0]:
pd_test_result[['tripid','prediction']].to_csv("/content/drive/My Drive/Colab Notebooks/FAIR_TAXI/result.csv", index=True)


In [0]:
# rf_random_pipe.best_estimator_